In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data# this contains our dataset
import numpy as np
from torch_geometric.datasets import Planetoid# this helps us manipulate the dataset
from torch_geometric.nn import GCNConv, SAGEConv, GATConv# this is a simple implementation of the GCN convolutional layer
import torch_geometric.transforms as T
from torch_geometric.utils import to_dense_adj
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

c:\Users\brown\miniconda3\envs\mainenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class GCN(torch.nn.Module):# this is the torch geometric implementation of our GCN model like before, it
    # is a lot simpler to implement and way customizeable
    def __init__(self, in_feat, hid_feat, out_feat):
        super().__init__()
        self.conv1 = GCNConv(in_feat, hid_feat)
        self.final = GCNConv(hid_feat, out_feat)
        self.activation = nn.ReLU()
        #self.dropout = nn.Dropout(p=.4)

    def forward(self, feats, edge_index):
        x = feats
        x1 = self.activation(self.conv1(x, edge_index))

        # Maybe get rid of dropout
        x2 = F.dropout(x1, training= self.training)

        x7 = self.final(x2, edge_index)
        return torch.mean(x7)

In [4]:
loss = nn.MSELoss()
num_feats = 10 #FIXME
num_hid = 16
output = 1
model = GCN(num_feats,num_hid,output).to("cuda:0")# this initializes our model and puts it on the GPU(change if you do not have a gpu)
optimizer = torch.optim.Adam(params=model.parameters(),lr = 5e-3, weight_decay=5e-4)# sets up our optimizer
train_feats = None # fill this with a torch Tensor (num_graphs x num_nodes x num_feats)
train_edge_index = None # fill this with a torch Tensor (num_graphs x num_edges x 2)
train_labels = None # fill this with a torch Tensor (num_graphs x 1)

val_feats = None # fill this with a torch Tensor
val_edge_index = None # fill this with a torch Tensor
val_labels = None # fill this with a torch Tensor

test_feats = None # fill this with a torch Tensor
test_edge_index = None # fill this with a torch Tensor
test_labels = None # fill this with a torch Tensor

In [12]:
loss(torch.Tensor([2]),torch.Tensor([3]))

tensor(1.)

In [ ]:
def accuracy(preds, labels):
    return torch.mean(torch.abs(preds - labels))

In [ ]:
model.train()# tells our model we are about to train
val_accs = []
train_accs = []

train_losses = []
val_losses = []
for epoch in range(100):# runs through all the data 200 times
    optimizer.zero_grad()
    train_preds = model(train_feats, train_edge_index)# runs our model on the data

    train_loss = loss(train_preds, train_labels)
    train_acc = accuracy(train_preds, train_labels)

    val_preds = model(val_feats, val_edge_index)

    val_loss = loss(val_preds, val_edge_index)
    val_acc = accuracy(val_preds, val_edge_index)

    if(epoch %5 == 0):
        print(f"epoch: {epoch} training_loss: {train_loss:.2} train_abs_error: {train_acc:.2} val_loss: {val_loss:.2} val_abs_error: {val_acc:.2}")
        
    train_loss.backward()
    optimizer.step()
    val_accs.append(val_acc)
    train_accs.append(train_acc)

    train_losses.append(train_loss.cpu().item())
    val_losses.append(val_loss.cpu().item())
test_preds = model(test_feats, test_edge_index)
test_loss = loss(test_preds, test_labels)
test_acc = accuracy(test_preds, test_labels)
print(f"test_loss: {test_loss:.2} test_abs_error: {test_acc:.2}")

In [ ]:
df_accs = pd.DataFrame({"train_acc":train_accs,"val_acc":val_accs})
sns.relplot(data=df_accs, kind = "line")
df_losses = pd.DataFrame({"train_loss":train_losses,"val_loss":val_losses})
sns.relplot(data=df_accs, kind = "line")
plt.show()